In [10]:
import librosa 
import librosa.display
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
path = Path.cwd()/'data'
new_path=Path.cwd()/'data_split'
answers = pd.read_csv('answers.csv')

In [5]:
whales = answers[answers['label']==1]
no_whales= answers[answers['label']==0]
no_whales_x_train, no_whales_x_val, no_whales_y_train, no_whales_y_val = train_test_split( no_whales[['image_path2']], no_whales.label, test_size=0.2, random_state=10)
whales_x_train, whales_x_val, whales_y_train, whales_y_val = train_test_split( whales[['image_path2','name']], whales.label, test_size=0.2, random_state=10)

In [263]:
# import subprocess

# for i, row in whales_x_train.iterrows():
#     subprocess.call(["cp", row[0], new_path/'training'/'whale' ])
    
# for i, row in whales_x_val.iterrows():
#     subprocess.call(["cp", row[0], new_path/'validation'/'whale'])    


In [265]:
# for i, row in no_whales_x_train.iterrows():
#     subprocess.call(["cp", row[0], new_path/'training'/'no_whale'])

In [266]:
# for i, row in no_whales_x_val.iterrows():
#     subprocess.call(["cp", row[0], new_path/'validation'/'no_whale'])

In [29]:
datagen = ImageDataGenerator()
images_directory = new_path
cb_training = datagen.flow_from_directory(images_directory/'training', class_mode='binary')
cb_validation = datagen.flow_from_directory(images_directory/'validation', class_mode='binary')


Found 23194 images belonging to 2 classes.
Found 3047 images belonging to 2 classes.


In [30]:
def import_data():
    datagen = ImageDataGenerator()
    new_path=Path.cwd()/'data_split'
    images_directory = new_path
    cb_training = datagen.flow_from_directory(images_directory/'training', class_mode='binary')
    cb_validation = datagen.flow_from_directory(images_directory/'validation', class_mode='binary')
    no_whales_x_train, no_whales_x_val, no_whales_y_train, no_whales_y_val = train_test_split( no_whales[['image_path2']], no_whales.label, test_size=0.2, random_state=10)
    whales_x_train, whales_x_val, whales_y_train, whales_y_val = train_test_split( whales[['image_path2','name']], whales.label, test_size=0.2, random_state=10)
    return (cb_training, whales_y_train, cb_validation, whales_y_val)


def base_set_up(filename, patience=2):
    
    xx =compute_class_weight(class_weight='balanced',classes=np.unique(answers.label), y=answers.label)
    class_weight = dict(zip(np.unique(answers.label), xx))
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    earlystop = EarlyStopping(monitor='val_accuracy',verbose=1, patience=patience)
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', save_best_only=True,save_weights_only=False, verbose=1)
    return (class_weight, METRICS, earlystop, checkpoint)

# CNN lets overtrain 

In [ ]:
cnn = None
keras.backend.clear_session()
filename = 'cnn_overfit2.h5'

cb_training, training_labels, cb_validation, val_labels  = import_data()
class_weight, METRICS, earlystop, checkpoint = base_set_up(filename, 10)

cnn = Sequential()
cnn.add(InputLayer(input_shape=(97,97, 3)))
cnn.add(Conv2D(filters=5, kernel_size=3, activation='relu', padding='same'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=10, kernel_size=3, activation='relu', padding='same'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=20, kernel_size=3, activation='relu', padding='same'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=30, kernel_size=3, activation='relu', padding='same'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=40, kernel_size=3, activation='relu', padding='same'))
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=50, kernel_size=3, activation='relu', padding='same'))
cnn.add(GlobalAveragePooling2D())

cnn.add(layers.Dense(100, activation='relu'))
cnn.add(layers.Dense(100, activation='relu'))
cnn.add(layers.Dense(100, activation='relu'))


cnn.add(layers.Dense(1, activation='sigmoid'))
optimizer = Adam(learning_rate=0.00001)

cnn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)

history = cnn.fit(cb_training, epochs=40,validation_data=cb_validation, class_weight = class_weight, callbacks=[earlystop, checkpoint])


Found 23194 images belonging to 2 classes.
Found 3047 images belonging to 2 classes.
Epoch 1/40
725/725 [==============================] - 235s 322ms/step - loss: 0.6317 - tp: 2261.0000 - fp: 4597.0000 - tn: 13781.0000 - fn: 2555.0000 - accuracy: 0.6916 - precision: 0.3297 - recall: 0.4695 - auc: 0.6585 - prc: 0.3218 - val_loss: 0.5562 - val_tp: 211.0000 - val_fp: 80.0000 - val_tn: 2264.0000 - val_fn: 492.0000 - val_accuracy: 0.8123 - val_precision: 0.7251 - val_recall: 0.3001 - val_auc: 0.8286 - val_prc: 0.6083

Epoch 00001: val_accuracy improved from -inf to 0.81227, saving model to cnn_overfit2.h5
Epoch 2/40
725/725 [==============================] - 244s 336ms/step - loss: 0.5810 - tp: 2994.0000 - fp: 4214.0000 - tn: 14164.0000 - fn: 1822.0000 - accuracy: 0.7398 - precision: 0.4154 - recall: 0.6217 - auc: 0.7675 - prc: 0.4406 - val_loss: 0.5366 - val_tp: 600.0000 - val_fp: 530.0000 - val_tn: 1814.0000 - val_fn: 103.0000 - val_accuracy: 0.7923 - val_precision: 0.5310 - val_recall: 0

725/725 [==============================] - 547s 755ms/step - loss: 0.3919 - tp: 4058.0000 - fp: 3848.0000 - tn: 14530.0000 - fn: 758.0000 - accuracy: 0.8014 - precision: 0.5133 - recall: 0.8426 - auc: 0.8909 - prc: 0.6395 - val_loss: 0.3898 - val_tp: 649.0000 - val_fp: 516.0000 - val_tn: 1828.0000 - val_fn: 54.0000 - val_accuracy: 0.8129 - val_precision: 0.5571 - val_recall: 0.9232 - val_auc: 0.9354 - val_prc: 0.7796

Epoch 00017: val_accuracy did not improve from 0.89038
Epoch 18/40
725/725 [==============================] - 574s 792ms/step - loss: 0.3842 - tp: 4073.0000 - fp: 3820.0000 - tn: 14558.0000 - fn: 743.0000 - accuracy: 0.8033 - precision: 0.5160 - recall: 0.8457 - auc: 0.8961 - prc: 0.6583 - val_loss: 0.2933 - val_tp: 603.0000 - val_fp: 305.0000 - val_tn: 2039.0000 - val_fn: 100.0000 - val_accuracy: 0.8671 - val_precision: 0.6641 - val_recall: 0.8578 - val_auc: 0.9387 - val_prc: 0.7908

Epoch 00018: val_accuracy did not improve from 0.89038
Epoch 19/40
725/725 [============

In [ ]:
hist = history.history
all_keys = list(hist.keys())

fig, axes = plt.subplots(5,2, figsize=(15,15))
for i,ax in enumerate(axes.flatten()):
        key = all_keys[i]
        val_key = 'val_'+key
        ax.plot(hist[key], label=key)
        ax.plot(hist[val_key], label = val_key)
        plt.xlabel('Epoch')
        plt.ylabel(key)
        ax.legend(loc='lower right')

In [ ]:
tf.math.confusion_matrix(cnn.predict(cb_validation), cb_validation.classes)


In [171]:
cnn.evaluate(cb_validation,verbose=2)


188/188 - 4s - loss: 0.1867 - tp: 1250.0000 - fp: 327.0000 - tn: 4274.0000 - fn: 149.0000 - accuracy: 0.9207 - precision: 0.7926 - recall: 0.8935 - auc: 0.9712 - prc: 0.9094


[0.18670053780078888,
 1250.0,
 327.0,
 4274.0,
 149.0,
 0.9206666946411133,
 0.7926442623138428,
 0.8934953808784485,
 0.971164882183075,
 0.9094194173812866]